In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [13]:
import re

def attributes(exp):    
    exp = exp.split("(")[1:]
    exp = "(".join(exp)
    exp = exp[:-1]
    exp = re.split("(?<!\(.),(?!.\))", exp)
    return exp

def initialPredicate(exp):
    return exp.split("(")[0]

def isConstant(char):
    return char.isupper() and len(char) == 1

def isVariable(char):
    return char.islower() and len(char) == 1

def replaceAttributes(exp, old, new):
    a = attributes(exp)
    for index, val in enumerate(a):
        if val == old:
            a[index] = new
    predicate = initialPredicate(exp)
    return predicate + "(" + ",".join(a) + ")"

def substitute(exp, subs):
    for sub in subs:
        new, old = sub[0], sub[2] 
        exp = replaceAttributes(exp, old, new)
    return exp

def occurence(var, exp):
    if exp.find(var) == -1:
        return False
    return True

def firstPart(exp):
    a = attributes(exp)
    return a[0]

def remainingPart(exp):
    predicate = initialPredicate(exp)
    a = attributes(exp)
    newExp = predicate + "(" + ",".join(a[1:]) + ")"
    return newExp

In [14]:
def unificationFOL(exp1, exp2):
    if exp1 == exp2:
        return []
    if isConstant(exp1) and isConstant(exp2):
        if exp1 != exp2:
            return False
    if isConstant(exp1):
        return [(exp1,'/', exp2)]
    if isConstant(exp2):
        return [(exp2,'/', exp1)]
    if isVariable(exp1):
        if occurence(exp1, exp2):        
            return False
        else:     
            return [(exp2,'/', exp1)]
    if isVariable(exp2):
        if occurence(exp2, exp1):
            return False
        else:
            return [(exp1,'/', exp2)]
    if initialPredicate(exp1) != initialPredicate(exp2):
        print("Predicates do not match. Cannot be unified")
        return False
    acount1 = len(attributes(exp1))
    acount2 = len(attributes(exp2))
    if acount1 != acount2:
        return False
    head1 = firstPart(exp1)
    print("Head1 ",head1)
    head2 = firstPart(exp2)
    print("Head2 ",head2)
    isub = unificationFOL(head1, head2)
    print("initial ",isub)   
    if not isub:
        return False
    if acount1 == 1:
        return isub
    tail1 = remainingPart(exp1)
    print("tail1 ",tail1)
    tail2 = remainingPart(exp2)
    print("tail2 ",tail2)
    if isub != []:
        tail1 = substitute(tail1, isub)
        print("tail11 ",tail1)
        tail2 = substitute(tail2, isub)
        print("tail21 ",tail2)
    rsub = unificationFOL(tail1, tail2)
    if not rsub:
        return False   
    isub.extend(rsub)
    return isub

In [15]:
def main():
    print("Enter expression 1: ")
    exp1 = input()
    print("Enter expression 2: ")
    exp2 = input()
    subs = unificationFOL(exp1, exp2)
    print("The substitutions needed are: ")
    print(subs)

In [20]:
#Testcase - 1
main()

Enter expression 1: 
King(John)
Enter expression 2: 
King(x)
Head1  John
Head2  x
initial  [('John', '/', 'x')]
The substitutions needed are: 
[('John', '/', 'x')]


In [21]:
#Testcase - 2
main()

Enter expression 1: 
Knows(F(x),y)
Enter expression 2: 
Knows(J,F(x))
Head1  F(x)
Head2  J
initial  [('J', '/', 'F(x)')]
tail1  Knows(y)
tail2  Knows(F(x))
tail11  Knows(y)
tail21  Knows(J)
Head1  y
Head2  J
initial  [('J', '/', 'y')]
The substitutions needed are: 
[('J', '/', 'F(x)'), ('J', '/', 'y')]


In [22]:
#Testcase - 3
main()

Enter expression 1: 
Knows(King(Richard))
Enter expression 2: 
Knows(Queen(x))
Head1  King(Richard)
Head2  Queen(x)
Predicates do not match. Cannot be unified
initial  False
The substitutions needed are: 
False
